In [1]:
#import base libraries
import numpy as np
import pandas as pd
import geopandas as gpd

#import dashboarding libraries
import panel as pn
pn.extension('tabulator')
import geoviews.tile_sources as gvts
import holoviews as hv
import hvplot.pandas
from holoviews import opts
from holoviews.plotting.links import DataLink

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
#reading in Charitable Ventures data - edited spreadsheet and city boundaries
CVsword = pd.read_csv("data/CV_CITY_CBO_KC_02.27.22.csv", encoding= 'unicode_escape')
cities = gpd.read_file("data/City_Boundaries.geojson")

In [ ]:
pd.set_option("display.max_columns",None) #expands number of viewable columns
CVsword = CVsword.fillna(0) #replaces NaNs with 0s to preserve all rows

In [ ]:
#define projection as CRS EPSG:6426 for CA zone 6
cities = cities.to_crs(epsg=6426)
cities = cities.drop(['JURISDICTI','Acres','Area_SqMi','OCSurveyDBOCityBoundariesArea'],axis=1) #drop extra columns

In [ ]:
#make join columns match
city_data = CVsword.rename(columns={'City':'CITY'})

#make join values match
city_data['CITY'] = city_data['CITY'].str.capitalize()
cities['CITY'] = cities['CITY'].str.capitalize()

In [ ]:
city_data.head()

In [ ]:
#join to city boundaries
city_data_joined = cities.merge(city_data, on='CITY', how='left').fillna(0)

#project
city_data_joined=gpd.GeoDataFrame(city_data_joined, geometry='geometry')
city_data_joined=city_data_joined.to_crs(epsg=6426)

In [ ]:
#isolating unincorporated rows to better generate unique city name list below
city_data_joined_Unincorporateds = city_data_joined.loc[city_data_joined['CITY']=='Unincorporated']
city_data_joined_incorporateds = city_data_joined.loc[city_data_joined['CITY']!='Unincorporated']

city_data_joined_incorporateds = city_data_joined_incorporateds.drop_duplicates(subset=['CITY'], keep='first')
city_data_joined = pd.concat([city_data_joined_incorporateds,city_data_joined_Unincorporateds])

In [ ]:
#read in cbo data in long form
CBO_data = pd.read_csv("data/CV_CITY_CBO_KC_2022feb27_long.csv", encoding = 'unicode_escape')
#drop unwanted columns by name
CBO_data = CBO_data.drop(['cbo_string','cbo_count'], axis=1)
#drop first unnamed column
CBO_data = CBO_data.iloc[:,1:]
#group by city column
CBOcount_byCity = CBO_data.groupby(['City']).count().reset_index()
#make join columns and values match
CBOcount_byCity = CBOcount_byCity.rename(columns={'City':'CITY'})
CBOcount_byCity['CITY'] = CBOcount_byCity['CITY'].str.capitalize()
#add column of cbo list
CBOcount_byCity = CBOcount_byCity.merge(city_data, on='CITY').rename(columns={'cbo_name_x':'Organizations', 'cbo_name_y':'cbo_names'})
#join to geometries
CBOcount_geo = cities.merge(CBOcount_byCity, on='CITY', how='left').fillna(0)
#drop unincorporated territories
CBOcount_geo = CBOcount_geo.loc[CBOcount_geo['CITY']!='Unincorporated']

In [ ]:
#make table for map
#rename column
CBOcount_geo=CBOcount_geo.rename(columns={'CITY':'City'})
citiesList = CBOcount_geo['City'].to_numpy()
cboList = CBOcount_geo['Organizations'].to_numpy()

table = hv.Table({'City':citiesList, 'CBO':cboList},
                ['City','CBO'])

In [ ]:
#make map
map00 = CBOcount_geo.hvplot(
c='Organizations',
frame_width=500,
frame_height=400,
dynamic=True,
geo=True,
crs=6426,
hover_cols=['City'],
cmap='Blues',
line_color='gray',
title='Number of Organizations by City')

basemap = gvts.CartoLight.opts(alpha=0.6)

map01=basemap*map00.opts(xaxis=None, yaxis=None)

In [ ]:
#data processing for org table
CVactivities= pd.read_csv("data/CVActivities_KC_02.26.22.csv", encoding= 'unicode_escape')
CVactivities = CVactivities.fillna("No Data")

In [ ]:
#convert to df
CVactivities = pd.DataFrame(CVactivities)

In [ ]:
#Table adjustments
#column names
CVactivities = CVactivities.rename(columns={'cbo_name':'Organization Name', 'activities':'Activities','HTC':'HTC Characteristic'})
#capitalize values
CVactivities['Language'] = CVactivities['Language'].str.title()
CVactivities['City'] = CVactivities['City'].str.title()

In [ ]:
#reading in csv with full HTC characteristics
fullHTCs = pd.read_csv('data/CVActivities_2022jan31_htc_categories_jm.csv')
#convert to df
fullHTC_df = pd.DataFrame(fullHTCs)
fullHTC_df = fullHTC_df[['cbo_name','htc_category','city_extracted']]
fullHTC_df = fullHTC_df.fillna("No Data")
fullHTC_df

In [ ]:
#generate list of just HTCs
allHTCs = ['born outside of the U.S.',
'children under 5',
'farmworker',
'houseless',
'immigrant / refugee',
'LGBTQ+',
'limited English speaking ability',
'low broadband subscription rate',
'older adult / senior',
'disabled',
'veteran'
]

allHTC_df = fullHTC_df.loc[fullHTC_df['htc_category'].isin(allHTCs)]

#first group by unique values, using x as dummy var to maintain df properties
fullHTC_byUniqueCBO = allHTC_df.groupby(['cbo_name', 'htc_category']).apply(list).reset_index(name='x')
uniqueHTC_byUniqueCBO = fullHTC_byUniqueCBO[['cbo_name','htc_category']] 

#then group by cbo name - using lambda to convert to concatenated string instead of list
uniqueHTC_byUniqueCBO = fullHTC_byUniqueCBO.groupby(['cbo_name'])['htc_category'].apply(lambda x: ','.join(x)).reset_index(name='HTC Characteristic')
uniqueHTC_byUniqueCBO = uniqueHTC_byUniqueCBO.fillna("No Data")
uniqueHTC_byUniqueCBO.head()

In [ ]:
#repeat for race/ethnic groups
allREs = ['Asian American & Pacific Islander (AAPI)',
'Black & African American',
'Latino/a/e',
'Middle Eastern & North African (MENA)',
'Native American & Tribal Communities'
]

allRE_df = fullHTC_df.loc[fullHTC_df['htc_category'].isin(allREs)]

#first group by unique values, using x as dummy var to maintain df properties
allRE_byUniqueCBO = allRE_df.groupby(['cbo_name', 'htc_category']).apply(list).reset_index(name='x')
uniqueRE_byUniqueCBO = allRE_byUniqueCBO[['cbo_name','htc_category']] 

#then group by cbo name - using lambda to convert to concatenated string instead of list
uniqueRE_byUniqueCBO = allRE_byUniqueCBO.groupby(['cbo_name'])['htc_category'].apply(lambda x: ','.join(x)).reset_index(name='Race/Ethnic Group')
uniqueRE_byUniqueCBO = uniqueRE_byUniqueCBO.fillna("No Data")
uniqueRE_byUniqueCBO.head()

In [ ]:
# add to main table 
#remove old HTCs
CVactivities = CVactivities[['Organization Name','City','Language','Activities']]

#make column names match
uniqueHTC_byUniqueCBO = uniqueHTC_byUniqueCBO.rename(columns={'cbo_name':'Organization Name'})
uniqueRE_byUniqueCBO = uniqueRE_byUniqueCBO.rename(columns={'cbo_name':'Organization Name'})
#join
CVactivities = CVactivities.merge(uniqueHTC_byUniqueCBO, on='Organization Name', how='left')
CVactivities = CVactivities.merge(uniqueRE_byUniqueCBO, on='Organization Name', how='left')
#replace Nans
CVactivities = CVactivities.fillna("No Data")
#readjust column order
#adjust column order
colOrder = ['Organization Name',
            'City',
            'Race/Ethnic Group',
            'Language',
            'HTC Characteristic',
            'Activities']
CVactivities = CVactivities[colOrder]
CVactivities

In [ ]:
filter_table=pn.widgets.Tabulator(CVactivities, 
                                  layout='fit_columns',
                                  pagination='remote',
                                  page_size=11,
                                  width=1200)

In [ ]:
#create city widget
cityNames_list = city_data_joined['CITY'].unique()
cityNames_arr = np.asarray(cityNames_list).astype('str')
cityNames = sorted(np.char.title(cityNames_arr).astype('O').tolist())

citySelect=pn.widgets.MultiSelect(options=cityNames, name='City Filter', height=300, )
filter_table.add_filter(citySelect, 'City')

#create file name box and download button
filename, button = filter_table.download_menu(
    text_kwargs={'name':'Enter filename', 'value':'OC_2020_Census_Activities.csv'},
    button_kwargs={'name':'Download table'}
)

In [ ]:
#create language widget
CVactivities_long = pd.read_csv("data/CVActivities_KC_2022feb22_long.csv")
language_list = CVactivities_long['language'].unique()
language_arr = np.asarray(language_list).astype('str')
languages = sorted(np.char.title(language_arr).astype('O').tolist())

langSelect=pn.widgets.MultiSelect(options=languages, name='Language Filter', height=300)
filter_table.add_filter(langSelect, 'Language')

In [ ]:
#creating type widgets
city_filter = pn.widgets.TextInput(name='Search by City Name', value='', width=200)
HTC_filter = pn.widgets.TextInput(name='Search by "Hard-to-Count" (HTC) Characteristic (copy from the list below)', value='', width=400)
RE_filter = pn.widgets.TextInput(name='Search by "Race/Ethnic Group (copy from the list below)', value='', width=330)
lang_filter = pn.widgets.TextInput(name='Search by Language', value='', width=200)

def contains_filter(df, pattern, column):
    if not pattern:
        return df
    return df[df[column].str.contains(pattern, na=False)]
    
filter_table.add_filter(pn.bind(contains_filter, pattern=city_filter, column='City'))
filter_table.add_filter(pn.bind(contains_filter, pattern=HTC_filter, column='HTC Characteristic'))
filter_table.add_filter(pn.bind(contains_filter, pattern=RE_filter, column='Race/Ethnic Group'))
filter_table.add_filter(pn.bind(contains_filter, pattern=lang_filter, column='Language'))

In [ ]:
with open('TITLE_ABOUT.md','r') as file:
    title = file.read()
    print(title)
    file.close()

In [ ]:
with open('RE.md','r') as file:
    REtext = file.read()
    print(REtext)
    file.close()

In [ ]:
with open('HTC.md','r') as file:
    HTCtext = file.read()
    print(HTCtext)
    file.close()

In [ ]:
with open('HTC2.md','r') as file:
    HTCtext2 = file.read()
    print(HTCtext2)
    file.close()

In [ ]:
with open('MAP.md','r') as file:
    mapText = file.read()
    print(mapText)
    file.close()

In [ ]:
with open('ORG_DETAILS.md','r') as file:
    orgDetailsText = file.read()
    print(orgDetailsText)
    file.close()

In [ ]:
with open('DOWNLOAD.md','r') as file:
    dlText = file.read()
    print(dlText)
    file.close()

In [ ]:
with open('BOTTOM.md','r') as file:
    bottomText = file.read()
    print(bottomText)
    file.close()

In [ ]:
pn.extension()

dash3 = pn.Column(title,
                pn.Row(map01, mapText),
                orgDetailsText,
                pn.Row(city_filter,
                         pn.Column(RE_filter,REtext),
                         lang_filter,
                         pn.Column(HTC_filter,
                         pn.Row(HTCtext,HTCtext2))),
                filter_table,
                dlText,
                filename,
                button,
                bottomText).servable()


In [ ]:
# run this to view in VSCode
# !pip install jupyter_bokeh

In [ ]:
dash3